In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

100.0%


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [5]:
test_data=datasets.FashionMNIST(
    root="data", 
    train=False, 
    download=True,
    transform=ToTensor()
)

In [7]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NN(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model = NN()

Hyperparameters are adjustable parameters that let you control the model optimization process. Different hyperparameter values can impact model training and convergence rates

In [8]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [9]:
loss_fn = nn.CrossEntropyLoss()

In [10]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [11]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    model.train()

    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [13]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done")


Epoch 1
-------------------------------
loss: 2.200746 [   64/60000]
loss: 2.192960 [ 6464/60000]
loss: 2.139698 [12864/60000]
loss: 2.139461 [19264/60000]
loss: 2.107677 [25664/60000]
loss: 2.057622 [32064/60000]
loss: 2.075549 [38464/60000]
loss: 2.012779 [44864/60000]
loss: 2.023936 [51264/60000]
loss: 1.941442 [57664/60000]
Test error: 
 Accuracy: 50.6%, Avg loss: 1.944863 

Epoch 2
-------------------------------
loss: 1.983381 [   64/60000]
loss: 1.957850 [ 6464/60000]
loss: 1.845559 [12864/60000]
loss: 1.864785 [19264/60000]
loss: 1.781670 [25664/60000]
loss: 1.728671 [32064/60000]
loss: 1.746397 [38464/60000]
loss: 1.652420 [44864/60000]
loss: 1.686060 [51264/60000]
loss: 1.563992 [57664/60000]
Test error: 
 Accuracy: 57.0%, Avg loss: 1.585509 

Epoch 3
-------------------------------
loss: 1.658868 [   64/60000]
loss: 1.621998 [ 6464/60000]
loss: 1.471075 [12864/60000]
loss: 1.521229 [19264/60000]
loss: 1.420999 [25664/60000]
loss: 1.411777 [32064/60000]
loss: 1.421521 [38464/